In [4]:
# CARGA DE TODOS LOS DATAFRAMES
import pandas as pd

df_equipos_1 = pd.read_csv('df_equipos_1.csv')
df_partidos_laliga_1 = pd.read_csv('df_partidos_laliga_1.csv')
df_partidos_todos_1 = pd.read_csv('df_partidos_todos_1.csv')
df_completo_1 = pd.read_csv('df_completo_1.csv')
df_proximos_partidos_1_openrefine = pd.read_csv('df_proximos_partidos_1_openrefine.csv')
df_proximos_partidos_1 = pd.read_csv('df_proximos_partidos_1.csv')
df_clasificacion_1 = pd.read_csv('df_clasificacion_1.csv')

df_equipos_2 = pd.read_csv('df_equipos_2.csv')
df_partidos_laliga_2 = pd.read_csv('df_partidos_laliga_2.csv')
df_partidos_todos_2 = pd.read_csv('df_partidos_todos_2.csv')
df_completo_2 = pd.read_csv('df_completo_2.csv')
df_proximos_partidos_2_openrefine = pd.read_csv('df_proximos_partidos_2_openrefine.csv')
df_proximos_partidos_2 = pd.read_csv('df_proximos_partidos_2.csv')
df_clasificacion_2 = pd.read_csv('df_clasificacion_2.csv')

In [5]:
#CARGA FUNCIONES PARA UN PARTIDO
import numpy as np
import pandas as pd
import warnings

def calcular_racha_detallada(equipo, df_partidos):
    """
    Calcula racha con LIMPIEZA AGRESIVA de duplicados.
    Normaliza fechas y nombres antes de filtrar para asegurar que borra los repetidos reales.
    """
    # 1. Filtramos partidos del equipo
    df_equipo = df_partidos[
        (df_partidos['Local'] == equipo) | 
        (df_partidos['Visitante'] == equipo)
    ].copy()
    
    # LIMPIEZA DE DATOS PREVIA AL FILTRADO
    
    # A) Normalizamos Nombres (Minúsculas y sin espacios extra)
    # Esto hace que "Girona" sea igual a "girona "
    df_equipo['Local_temp'] = df_equipo['Local'].astype(str).str.lower().str.strip()
    df_equipo['Visitante_temp'] = df_equipo['Visitante'].astype(str).str.lower().str.strip()
    
    # B) Normalizamos Fecha (Extraemos solo dd.mm.yyyy)
    # Esto hace que "Sáb 20.01.2024" sea igual a "20.01.2024"
    if 'Fecha' in df_equipo.columns:
        df_equipo['Fecha_temp'] = df_equipo['Fecha'].astype(str).str.extract(r'(\d{2}\.\d{2}\.\d{4})')
    else:
        df_equipo['Fecha_temp'] = "sin_fecha"

    # BORRADO DE DUPLICADOS
    # Usamos las columnas normalizadas (_temp) como criterio
    df_equipo = df_equipo.drop_duplicates(subset=['Fecha_temp', 'Local_temp', 'Visitante_temp'], keep='first')
    
    # 2. CONVERSIÓN DE FECHA REAL
    # Usamos la fecha limpia que acabamos de crear
    if 'Fecha_temp' in df_equipo.columns:
        df_equipo['Fecha_dt'] = pd.to_datetime(df_equipo['Fecha_temp'], format='%d.%m.%Y', errors='coerce')
        df_equipo = df_equipo.dropna(subset=['Fecha_dt'])
        df_equipo = df_equipo.sort_values('Fecha_dt', ascending=False)
    else:
        df_equipo = df_equipo.sort_index(ascending=False)

    # 3. Cogemos los 5 más recientes (ya limpios y únicos)
    partidos_recientes = df_equipo.head(5)
    
    puntos = 0
    historial_letras = [] 
    rivales = [] 
    
    if partidos_recientes.empty:
        return 0, 1.0, "Sin datos", ""

    for _, partido in partidos_recientes.iterrows():
        try:
            g_loc = float(partido['Goles_local'])
            g_vis = float(partido['Goles_visitante'])
        except ValueError:
            continue
            
        if partido['Local'] == equipo:
            favor, contra = g_loc, g_vis
            rival = partido['Visitante']
        else:
            favor, contra = g_vis, g_loc
            rival = partido['Local']
            
        if favor > contra:
            puntos += 3
            historial_letras.append("G")
        elif favor == contra:
            puntos += 1
            historial_letras.append("E")
        else:
            historial_letras.append("P")
            
        fecha_str = partido['Fecha_dt'].strftime('%d/%m') if 'Fecha_dt' in partido else ""
        rivales.append(f"vs {rival} ({fecha_str})")
            
    # 4. Invertir para display
    texto_racha = "-".join(reversed(historial_letras))
    texto_rivales = " | ".join(reversed(rivales))
    
    # 5. Cálculo del factor de forma
    # Fórmula: 0.90 + (puntos obtenidos en últimos 5 partidos / 15) * 0.25
    # Explicación: Mínimo 0.90 (0 puntos), máximo 1.15 (15 puntos). Sólo varía un 25%.
    factor_forma = 0.90 + (puntos / 15) * 0.25
    
    return puntos, factor_forma, texto_racha, texto_rivales

def obtener_posicion(equipo, df_clasificacion):
    fila = df_clasificacion[df_clasificacion['Equipo'] == equipo]
    if fila.empty: 
        return 10 # Valor por defecto si no se encuentra
    if 'Pos' in df_clasificacion.columns: 
        pos = fila['Pos'].values[0]
        pos = pos.replace('º', '').replace('ª', '') # Limpieza
        return int(pos)
    elif 'Posicion' in df_clasificacion.columns: 
        pos = fila['Posicion'].values[0]
        pos = str(pos).replace('º', '').replace('ª', '') # Limpieza
        return int(pos)
    else: 
        return df_clasificacion.index.get_loc(fila.index[0]) + 1

def predecir_resultado_final(equipo_local, equipo_visitante, df_completo, df_partidos, df_clasificacion):
    
    # 1. VALIDACIÓN
    equipos_validos = df_completo['Equipo'].unique().tolist()
    if equipo_local not in equipos_validos or equipo_visitante not in equipos_validos:
        print(f"Error: Revisa los nombres. Disponibles: {sorted(equipos_validos)}")
        return None

    # 2. EXTRACCIÓN DE DATOS
    dl = df_completo[df_completo['Equipo'] == equipo_local]
    dv = df_completo[df_completo['Equipo'] == equipo_visitante]
    
    valor_real_l = dl['Valor_Mercado'].iloc[0]
    valor_real_v = dv['Valor_Mercado'].iloc[0]
    puntos_liga_l = dl['Puntos'].iloc[0]
    puntos_liga_v = dv['Puntos'].iloc[0]
    les_l = dl['Lesionados'].iloc[0]
    les_v = dv['Lesionados'].iloc[0]
    
    val_l_log = np.log(valor_real_l) 
    val_v_log = np.log(valor_real_v)
    
    puntos_racha_l, factor_forma_l, txt_racha_l, txt_rivales_l = calcular_racha_detallada(equipo_local, df_partidos)
    puntos_racha_v, factor_forma_v, txt_racha_v, txt_rivales_v = calcular_racha_detallada(equipo_visitante, df_partidos)
    
    pos_l = obtener_posicion(equipo_local, df_clasificacion)
    pos_v = obtener_posicion(equipo_visitante, df_clasificacion)

    # 3. ESTADÍSTICAS DE GOLES
    # Partidos jugados
    pj_l = dl['Partidos_jugados'].iloc[0]
    pj_v = dv['Partidos_jugados'].iloc[0]
    
    # Medias de goles
    media_gf_l = dl['Goles_favor'].iloc[0] / pj_l
    media_gc_l = dl['Goles_contra'].iloc[0] / pj_l
    media_gf_v = dv['Goles_favor'].iloc[0] / pj_v
    media_gc_v = dv['Goles_contra'].iloc[0] / pj_v
    
    # 4. CÁLCULO DE PREDICCIÓN (LÓGICA RAZONADA)
    # Base xG antes de ajustes
    # Cálculo: media de las medias goles a favor propio + medias goles en contra rival
    xg_base_local = (media_gf_l + media_gc_v) / 2
    xg_base_visita = (media_gf_v + media_gc_l) / 2
    
    # Ajustes Contextuales
    # A. Valor Mercado
    # Si hay una diferencia muy grande, se ajusta a favor del más caro
    if valor_real_l > valor_real_v * 2: 
        xg_base_local *= 1.15
        xg_base_visita *= 0.90
    elif valor_real_v > valor_real_l * 2:
        xg_base_local *= 0.90
        xg_base_visita *= 1.15
        
    # B. Forma y Campo
    # Local tiene ventaja de campo
    xg_base_local *= (factor_forma_l * 1.20)
    xg_base_visita *= (factor_forma_v * 0.85)
    
    # C. Lesiones
    # Penaliza un 2% por cada baja confirmada
    xg_base_local *= (1 - (les_l * 0.02))
    xg_base_visita *= (1 - (les_v * 0.02))

    # 5. RESULTADO FINAL
    # Cálculo de probabilidades
    # Fórmula ponderada: (xG * 100) + (puntos de liga * 0.5)
    pi_local = xg_base_local * 100 + (puntos_liga_l * 0.5)
    pi_visita = xg_base_visita * 100 + (puntos_liga_v * 0.5)
    
    total = pi_local + pi_visita
    # La probabilidad en porcentaje
    prob_local = (pi_local / total) * 100
    prob_visita = (pi_visita / total) * 100
    
    # Ajuste de probabilidad de empate según diferencia
    diferencia = abs(prob_local - prob_visita)
    prob_empate = 25 if diferencia < 10 else (15 if diferencia < 20 else 5)
    
    prob_local -= prob_empate/2
    prob_visita -= prob_empate/2
    
    # Redondeo de goles finales
    goles_l_final = int(xg_base_local)
    goles_v_final = int(xg_base_visita)
    
    dec_l = xg_base_local - goles_l_final
    dec_v = xg_base_visita - goles_v_final
    
    # Si la parte decimal es alta o la probabilidad de victoria es alta, sumamos un gol más
    if dec_l > 0.55 or (dec_l > 0.40 and prob_local > 50): goles_l_final += 1
    if dec_v > 0.55 or (dec_v > 0.40 and prob_visita > 50): goles_v_final += 1
    
    # 3. BLOQUE DE COHERENCIA INTELIGENTE (+1 o -1)
    # Verificamos si hay contradicción fuerte entre Probabilidad y Goles
    # Umbral: Diferencia de probabilidad > 10%
    
    # CASO A: Favorito LOCAL pero marcador dice Empate/Derrota
    if prob_local > (prob_visita + 10) and goles_l_final <= goles_v_final:
        # ¿Qué es más probable? ¿Que el Local marque otro o que el Visitante no marque?
        
        # Si el visitante tenía muy pocos méritos (ej: xG 0.65 -> redondeado a 1)
        # es más justo RESTARLE el gol (1-0) que sumar al local (2-1).
        if dec_v < 0.70 and goles_v_final > 0:
            goles_v_final -= 1 # La defensa local anula el gol visitante
        else:
            goles_l_final += 1 # El ataque local fuerza un gol extra
            
    # CASO B: Favorito VISITANTE pero marcador dice Empate/Derrota
    elif prob_visita > (prob_local + 10) and goles_v_final <= goles_l_final:
        
        # Si el local tenía poco xG (ej: 0.65 -> 1), se lo quitamos
        if dec_l < 0.70 and goles_l_final > 0:
            goles_l_final -= 1 # La defensa visitante deja a cero al local
        else:
            goles_v_final += 1 # El ataque visitante marca uno más

    # Definir texto pronóstico final
    if goles_l_final > goles_v_final: pronostico = f"Gana {equipo_local}"
    elif goles_v_final > goles_l_final: pronostico = f"Gana {equipo_visitante}"
    else: pronostico = "Empate"

    # 6. SALIDA DE DATOS
    return {
        "1. CABECERA DEL PARTIDO": f"{equipo_local} (Pos: {pos_l}º) vs {equipo_visitante} (Pos: {pos_v}º)",
        
        "2. PREDICCIÓN PRINCIPAL": {
            "Resultado Esperado": pronostico,
            "Marcador Estimado": f"{goles_l_final} - {goles_v_final}",
            "Goles Esperados (xG)": f"{round(xg_base_local, 2)} - {round(xg_base_visita, 2)}",
            "Confianza Local": f"{equipo_local}: {round(prob_local, 1)}%",
            "Confianza Empate": f"{round(prob_empate, 1)}%",
            "Confianza Visitante": f"{equipo_visitante}: {round(prob_visita, 1)}%"
        },
        
        "3. MOMENTO DE FORMA (ÚLTIMOS 5)": {
            f"Racha {equipo_local}": f"{txt_racha_l} ({puntos_racha_l} puntos de 15)",
            f"Últimos Rivales {equipo_local}": txt_rivales_l,
            f"Racha {equipo_visitante}": f"{txt_racha_v} ({puntos_racha_v} puntos de 15)",
            f"Últimos Rivales {equipo_visitante}": txt_rivales_v,
            "Tendencia": f"{equipo_local} llega mejor" if puntos_racha_l > puntos_racha_v else (f"{equipo_visitante} llega mejor" if puntos_racha_v > puntos_racha_l else "Llegan igualados")
        },
        
        "4. DATOS DE LA TEMPORADA": {
            "Puntos en Liga": f"{puntos_liga_l} vs {puntos_liga_v}",
            "Media Goles A Favor": f"{round(media_gf_l, 2)} vs {round(media_gf_v, 2)}",
            "Media Goles En Contra": f"{round(media_gc_l, 2)} vs {round(media_gc_v, 2)}"
        },
        
        "5. PLANTILLA Y BAJAS": {
            "Valor de Mercado": f"{valor_real_l:,.1f} M€ vs {valor_real_v:,.1f} M€",
            "Diferencia": f"x{round(valor_real_l/valor_real_v, 1) if valor_real_l > valor_real_v else round(valor_real_v/valor_real_l, 1)} a favor de {equipo_local if valor_real_l > valor_real_v else equipo_visitante}",
            "Bajas confirmadas": f"{equipo_local}: {les_l} vs {equipo_visitante}: {les_v}"
        }
    }

#CARGA FUNCIONES PARA PRÓXIMOS PARTIDOS Y JORNADA
import pandas as pd
import numpy as np

def obtener_pred_proximos_partidos(df_proximos_partidos):
    """
    Busca en el dataframe los partidos que no tienen resultado (goles = NaN o vacío).
    Devuelve una lista de tuplas [(Local, Visitante, Fecha), ...].
    """
    pendientes = []
    
    # 1. Limpieza de fechas para ordenar
    # (Usamos lógica de limpieza para poder ordenar por fecha más próxima)
    if 'Fecha' in df_proximos_partidos.columns:
        df_proximos_partidos['Fecha_clean'] = df_proximos_partidos['Fecha'].astype(str).str.extract(r'(\d{2}\.\d{2}\.\d{4})')
        df_proximos_partidos['Fecha_dt'] = pd.to_datetime(df_proximos_partidos['Fecha_clean'], format='%d.%m.%Y', errors='coerce')
        df_proximos_partidos = df_proximos_partidos.sort_values('Fecha_dt', ascending=True) # Ascendente: los más próximos primero
    
    # 2. Extraemos las parejas únicas
    # Usamos un set para no repetir (por si el scraping tiene duplicados)
    vistos = set()
    
    for _, fila in df_proximos_partidos.iterrows():
        local = str(fila['Local']).strip()
        visita = str(fila['Visitante']).strip()
        fecha = str(fila['Fecha']).strip()
        
        # Clave única del partido
        clave = (local, visita)
        
        if clave not in vistos:
            pendientes.append((local, visita, fecha))
            vistos.add(clave)
            
    return pendientes

def obtener_reporte_jornada(df_completo, df_partidos_laliga, df_clasificacion, df_proximos_partidos):
    """
    Genera un DataFrame con las predicciones de todos los partidos pendientes encontrados.
    """
    proximos_partidos = obtener_pred_proximos_partidos(df_proximos_partidos)
    
    if not proximos_partidos:
        print("No se encontraron partidos pendientes (sin resultado) en df_proximos_partidos.")
        print("Asegúrate de que df_proximos_partidos contiene el calendario futuro.")
        return None
    
    resultados_lista = []
    
    for local, visita, fecha in proximos_partidos:
        # Ejecutamos función de predicción
        # Usamos try/except para que si falla un equipo (por nombre), no pare todo el proceso
        try:
            datos = predecir_resultado_final(local, visita, df_completo, df_partidos_laliga, df_clasificacion)
            
            if datos:
                # Extraemos los datos limpios del diccionario complejo que retorna tu función
                pred = datos["2. PREDICCIÓN PRINCIPAL"]
                
                fila = {
                    "Fecha": fecha,
                    "Partido": f"{local} vs {visita}",
                    "Pronóstico": pred["Resultado Esperado"],
                    "Marcador (Est)": pred["Marcador Estimado"],
                    "xG (Esperados)": pred["Goles Esperados (xG)"],
                    "Confianza Local": pred["Confianza Local"],
                    "Confianza Empate": pred["Confianza Empate"],
                    "Confianza Visitante": pred["Confianza Visitante"]
                }
                resultados_lista.append(fila)
            
        except Exception as e:
            print(f"Error procesando {local} vs {visita}: {e}")

    # Convertimos a DataFrame bonito
    df_resultados = pd.DataFrame(resultados_lista)
    return df_resultados

df_predicciones_jornada_1 = obtener_reporte_jornada(df_completo_1, df_partidos_laliga_1, df_clasificacion_1, df_proximos_partidos_1)
df_predicciones_jornada_2 = obtener_reporte_jornada(df_completo_2, df_partidos_laliga_2, df_clasificacion_2, df_proximos_partidos_2)

In [ ]:
# SERVIDOR SOCKET PARA ENVIAR DATOS
# (CLIENTE SOCKET EN Cliente.ipynb)

from socket import *
import pickle # Necesario para enviar el DataFrame (objetos Python)

# Aseguramos que el DataFrame a enviar está definido
if df_predicciones_jornada_1 is not None:
    df_a_enviar_1 = df_predicciones_jornada_1
if df_predicciones_jornada_2 is not None:
    df_a_enviar_2 = df_predicciones_jornada_2


def iniciar_servidor_clase():
    # Definimos puerto
    serverPort = 12000 
    
    # Creamos el socket TCP (AF_INET, SOCK_STREAM)
    serverSocket = socket(AF_INET, SOCK_STREAM)
    
    # Enlazamos el socket al puerto (bind)
    serverSocket.bind(('', serverPort))
    
    # El servidor comienza a escuchar
    serverSocket.listen(1)
    
    print('The server is ready to receive (Esperando cliente...)')
    
    while True:
        # Esperamos conexión (accept)
        connectionSocket, addr = serverSocket.accept()
        print(f"Conexión aceptada desde: {addr}")
        
        try:
            message_str = connectionSocket.recv(1024).decode()
            print(f"Mensaje del cliente: {message_str}")
        
            # Procesamos el mensaje recibido
            partes = message_str.split('|')
            comando = partes[0]

            if comando == "DAME_PREDICCIONES_PARTIDO":
                division = partes[1]
                local = partes[2]
                visitante = partes[3]
                if division == 'PRIMERA':
                    datos_prediccion = predecir_resultado_final(local, visitante, df_completo_1, df_partidos_laliga_1, df_clasificacion_1)
                elif division == 'SEGUNDA':
                    datos_prediccion = predecir_resultado_final(local, visitante, df_completo_2, df_partidos_laliga_2, df_clasificacion_2)
                else:
                    datos_prediccion = {"Error": "División no válida. Usa 'PRIMERA' o 'SEGUNDA'."}
                
                # Preparamos los datos (Serialización)
                datos_serializados = pickle.dumps(datos_prediccion)

                #Enviamos los datos por el socket (send) (diccionario)
                connectionSocket.sendall(datos_serializados)
                print("Datos enviados correctamente.")
            
            if comando == "DAME_PREDICCIONES_PRIMERA":
                # Preparamos los datos (Serialización)
                datos_serializados = pickle.dumps(df_predicciones_jornada_1)
                
                #Enviamos los datos por el socket (send) (df)
                connectionSocket.sendall(datos_serializados)
                print("Datos enviados correctamente.")
            
            if comando == "DAME_PREDICCIONES_SEGUNDA":
                # Preparamos los datos (Serialización)
                datos_serializados = pickle.dumps(df_predicciones_jornada_2)
                
                #Enviamos los datos por el socket (send) (df)
                connectionSocket.sendall(datos_serializados)
                print("Datos enviados correctamente.")
                
        except Exception as e:
            print(f"Error: {e}")
            
        # Cerramos el socket de la conexión
        connectionSocket.close()
        print("Conexión cerrada. Esperando nuevo cliente...\n")


# Ejecutar solo si hay datos
if 'df_predicciones_jornada_1' in locals() and df_predicciones_jornada_1 is not None and 'df_predicciones_jornada_2' in locals() and df_predicciones_jornada_2 is not None:
    iniciar_servidor_clase()
else:
    print("Primero debes ejecutar el código de predicciones para tener datos que enviar.")

The server is ready to receive (Esperando cliente...)
Conexión aceptada desde: ('127.0.0.1', 60301)
Mensaje del cliente: DAME_PREDICCIONES_PARTIDO|PRIMERA|Real Betis|Valencia CF
Datos enviados correctamente.
Conexión cerrada. Esperando nuevo cliente...

